# Import Library

In [ ]:
import unicodedata

import torch
import pandas as pd
from tqdm import tqdm

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig,
)

# Langchain 관련
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate 
from langchain.schema.output_parser import StrOutputParser

# CONFIG

In [ ]:
class CFG:
    # Root of Files
    EMBEDDING_CSV_TEST = './datas/stf_e5_base_test.csv'

    SAVING_FINETUNING_MODEL_DIR = "./finetune_models/gemma_ko_9b_ver1.01"

    PIPELINE_TEMPERATURE=0.2
    SEQ_MAX_LENGHT=450

    # About Submission
    SAMPLE_SUBMISSION = './sample_submission.csv'
    RESULT_SUBMISSION = './gemma_9b_ver1.01_submission.csv'


### Loading Finetuned Model (for Quantization) and Make Pipeline

In [ ]:
def setup_llm_SFTTrainer_with_finetuning():
    # 4비트 양자화 설정
    bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
    )
    
    # loading model and tokenizer
    model = AutoModelForCausalLM.from_pretrained(CFG.SAVING_FINETUNING_MODEL_DIR, 
                                                 quantization_config=bnb_config,
                                                 trust_remote_code=True,
                                                 device_map="auto",)
    tokenizer = AutoTokenizer.from_pretrained(CFG.SAVING_FINETUNING_MODEL_DIR)

    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        temperature=CFG.PIPELINE_TEMPERATURE,
        return_full_text=False,
        max_new_tokens=CFG.SEQ_MAX_LENGHT, 
    )

    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)
    return hf

In [ ]:
llm = setup_llm_SFTTrainer_with_finetuning()

# EVALUATION

### Utils for Evaluation

In [ ]:
def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for doc in docs:
        context += doc.page_content
        context += '\n'
    return context

### evaluation

In [ ]:
# 결과를 저장할 리스트 초기화
results = []

# CSV 파일 읽기
df = pd.read_csv(CFG.EMBEDDING_CSV_TEST)

# DataFrame의 각 행에 대해 처리
for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
    # 소스 문자열 정규화
    context = normalize_string(row['Context'])
    question = normalize_string(row['Question'])

    # RAG 체인 구성
    template = """다음 정보를 바탕으로 질문에 답하세요. 답변은 꼭 문장으로 하세요. 주어를 꼭 적으세요. :
    {context}

    질문: {question}

    답변:
    """
    prompt = PromptTemplate.from_template(template)

    # RAG 체인 정의
    rag_chain = ( 
        prompt
        | llm
        | StrOutputParser()
    )

    # 답변 추론
    # print(f"Question: {question}")
    full_response = rag_chain.invoke({"context":context, "question":question})

    # print(f"Answer: {full_response}\n")
    
    # 결과 저장
    results.append({
        'Question': question,
        'Context': context,
        'Answer': full_response
    })

# SUBMISSIOIN

In [ ]:
# 제출용 샘플 파일 로드
submit_df = pd.read_csv(CFG.SAMPLE_SUBMISSION)

# 생성된 답변을 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'] for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

# 결과를 CSV 파일로 저장
submit_df.to_csv(CFG.RESULT_SUBMISSION, encoding='UTF-8-sig', index=False)